In [1]:
import pandas as pd

# Load train and test data
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")

# Split into features and labels
X_train = train_data.drop(columns=["Anomaly_Label"])
y_train = train_data["Anomaly_Label"]

X_test = test_data.drop(columns=["Anomaly_Label"])
y_test = test_data["Anomaly_Label"]

print(f"Loaded Training Set: {X_train.shape}, Testing Set: {X_test.shape}")


Loaded Training Set: (936, 12), Testing Set: (235, 12)


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
y_pred_iso = iso_forest.fit_predict(X_train_scaled)

# Convert predictions (-1 for anomalies, 1 for normal) to (0,1) format
y_pred_iso = [1 if x == -1 else 0 for x in y_pred_iso]

# Evaluate Model
from sklearn.metrics import classification_report
print("Isolation Forest Classification Report:")
print(classification_report(y_train, y_pred_iso))


Isolation Forest Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.84      0.66       542
           1       0.10      0.02      0.04       394

    accuracy                           0.50       936
   macro avg       0.32      0.43      0.35       936
weighted avg       0.35      0.50      0.40       936



In [3]:
print("X_train shape:", X_train.shape)


X_train shape: (936, 12)


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

# Normalize Data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for LSTM (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build LSTM Model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True, input_shape=(1, X_train.shape[2]))),
    BatchNormalization(),
    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=False)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile Model
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])

# Train Model with Learning Rate Scheduler
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[reduce_lr])

# Evaluate Model
y_pred = (model.predict(X_test) > 0.5).astype('int32')
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))




Epoch 1/50


C:\Users\MADHUCHANDRA\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step - accuracy: 0.5439 - loss: 0.7939 - val_accuracy: 0.6255 - val_loss: 0.6777 - learning_rate: 5.0000e-04
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6741 - loss: 0.6463 - val_accuracy: 0.6255 - val_loss: 0.6710 - learning_rate: 5.0000e-04
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6886 - loss: 0.6316 - val_accuracy: 0.6255 - val_loss: 0.6669 - learning_rate: 5.0000e-04
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6905 - loss: 0.5789 - val_accuracy: 0.6255 - val_loss: 0.6653 - learning_rate: 5.0000e-04
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7436 - loss: 0.5135 - val_accuracy: 0.6255 - val_loss: 0.6644 - learning_rate: 5.0000e-04
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7125 - loss: 0.5132 - val_accuracy: 0.6255 - val_loss: 0.6655 - learning_rate: 5.0000e-04
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7993 - loss: 0.